## Init

In [3]:
import duckdb
import pandas as pd
from sql.ggplot import ggplot, aes, geom_boxplot, geom_histogram, facet_wrap

# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

%reload_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///MIMIC_IV.duckdb

In [13]:
%%sql

create schema if not exists datathon

,Success


## Cohort building

- 30 Day Unplanned Hospital-wide All-Cause Readmission Algorithm
    - step 0 convert icd 10 dx and pr to ccs
    - step 1 Index Admission Algorithm
    - step 2 Planned Admission Algorithm
    - step 3 Unplanned Readmission Algorithm
- step 3 Social work referral flag
- Other covariates
    - step 3 social demographics
    - step 4 charlson comorbidity index




### 0 convert icd 10 dx and pr to ccs
NOTE: the created tables datathon.hosp_diagnoses and datathon.hosp_procedures are filtered by icd 10 only.


quality check: only 1989449 - 1988940 = 509 are not matched

In [21]:
%%sql
select count(*)
from hosp.diagnoses_icd
where icd_version = 10


,count_star()
0,1989449


In [14]:
%%sql
drop table if exists datathon.hosp_diagnoses;
create table datathon.hosp_diagnoses as
select hd.*, tr.description as icd_10_desc, tr.ccs_diagnosis_category, tr.ccs_description
from hosp.diagnoses_icd hd
    inner join tuva_readm.readmissions__icd_10_cm_to_ccs tr on hd.icd_code = tr.icd_10_cm
where hd.icd_version = 10

,Success


quality check: only 223107 - 223107 = 0 are not matched

In [19]:
%%sql
select count(*)
from hosp.procedures_icd
where icd_version = 10

,count_star()
0,223107


In [25]:
%%sql
select count(*)
from hosp.procedures_icd hp
    inner join tuva_readm.readmissions__icd_10_pcs_to_ccs tr on hp.icd_code = tr.icd_10_pcs
where hp.icd_version = 10

,count_star()
0,223107


In [26]:
%%sql

drop table if exists datathon.hosp_procedures;
create table datathon.hosp_procedures as
select hp.*, tr.description as icd_10_desc, tr.ccs_procedure_category, tr.ccs_description
from hosp.procedures_icd hp
    inner join tuva_readm.readmissions__icd_10_pcs_to_ccs tr on hp.icd_code = tr.icd_10_pcs
where hp.icd_version = 10

,Success


### 1 Index Admission Algorithm

Here are the sub-algorithms used by the Hospital-wide Measure:

- 1.1 Cohort Inclusion Algorithm
- 1.2 Cohort Exclusion Algorithm
- 1.3 Discharged Alive and Not Against Medical Advice
- 1.4 Not a Transfer
- 1.5 Not a Same Day Admission
- 1.6 Run-out

If an inpatient admission meets the criteria from all of these sub-algorithms then it qualifies as an index admission. Otherwise it does not. Below we walk through each sub-algorithm.

#### 1.1 Cohort Inclusion Algorithm

The cohort inclusion algorithm is a set of rules that determine whether an inpatient admission belongs to 1 or more specialty categories. In order to qualify as an index admission, an inpatient admission must belong to 1 or more of these 5 specialty categories:

- Medicine
- Cardiorespiratory
- Cardiovascular
- Neurology
- Surgery / Gynecology

Here are the steps to implement the cohort inclusion algorithm:

1. Each inpatient admission should have **1 primary** ICD-10-CM diagnosis code and 0 or more ICD-10-PCS procedure codes associated with it.

2. Map each of these codes to the custom CCS Condition Categories (terminology datasets #8 and #9). Now each inpatient admission should have its corresponding CCS diagnosis and procedure categories assigned.

3. For each inpatient admission, map its assigned CCS diagnosis and procedure categories to those listed in the HWR Specialty Cohort Inclusion terminology dataset (terminology dataset #1) to determine if it should be assigned to **any of the first 4** specialty categories (medicine, cardiorespiratory, cardiovascular, and neurology).

4. For each inpatient admission, map its ICD-10-PCS codes to those listed in the HWR Surg-Gyn Cohort Inclusion terminology dataset (terminology dataset #2) to determine if the readmission should be assigned to the Surgical / Gynecology specialty category.

5. Inpatient admissions are successfully assigned to **one or more specialty categories** based on steps 3 and 4 pass this algorithm. Otherwise the admission fails and is no longer a candidate for an index admission.

In [80]:
%%sql
drop table if exists datathon.step_1_1_incl_cr ;
create table datathon.step_1_1_incl_cr as
with adm_one_icd10 as (
    select ha.*
    from hosp.admissions ha 
        inner join datathon.hosp_diagnoses dh on ha.hadm_id = dh.hadm_id and dh.seq_num = 1
),

specialty4_dx_code as (
    select *
    from tuva_readm.readmissions__specialty_cohort
    where procedure_or_diagnosis = 'Diagnosis' and specialty_cohort 
        in ('Medicine', 'Cardiorespiratory', 'Neurology', 'Cardiovascular')
),

specialty4_pcs_code as (
    select *
    from tuva_readm.readmissions__specialty_cohort
    where procedure_or_diagnosis = 'Procedure' and specialty_cohort 
        in ('Medicine', 'Cardiorespiratory', 'Neurology', 'Cardiovascular')
),

primary_dx as (

    select *
    from datathon.hosp_diagnoses
    where seq_num = 1

),

adm_surg_gyn as (
    select distinct adm_one_icd10.*, specialty_cohort
    from adm_one_icd10
        join datathon.hosp_procedures dh on adm_one_icd10.hadm_id = dh.hadm_id
        join tuva_readm.readmissions__surgery_gynecology_cohort  tr
            on tr.icd_10_pcs = dh.icd_code
),

adm_5cat as (
    select distinct adm_one_icd10.*, specialty_cohort
    from adm_one_icd10
        join primary_dx on adm_one_icd10.hadm_id = primary_dx.hadm_id
        join specialty4_dx_code 
            on specialty4_dx_code.ccs = primary_dx.ccs_diagnosis_category
    union
    select distinct adm_one_icd10.*, specialty_cohort
    from adm_one_icd10
        join datathon.hosp_procedures dh on adm_one_icd10.hadm_id = dh.hadm_id
        join specialty4_pcs_code 
            on specialty4_pcs_code.ccs = dh.ccs_procedure_category
    union 
    select *
    from adm_surg_gyn
    ),

adm_5cat_one_cat_id as (
    select hadm_id
    from adm_5cat
    group by hadm_id
    having count(*) = 1
),
 
adm_5cat_onemore_cat_id as (
    select hadm_id
    from adm_5cat
    group by hadm_id
    having count(*) > 1
),

adm_5cat_unique as (
    select a.*
    from adm_5cat a
        join adm_5cat_one_cat_id b on a.hadm_id = b.hadm_id
    union
    select a.* EXCLUDE (specialty_cohort), 'More' as specialty_cohort
    from adm_5cat a
        join adm_5cat_onemore_cat_id b on a.hadm_id = b.hadm_id

)

-- no duplicate, checked below

-- select hadm_id, count(*)
-- from adm_5cat_unique
-- group by hadm_id
-- having count(*) > 1

select *
from adm_5cat_unique



,Success


#### 1.2 Cohort Exclusion Algorithm

An inpatient admission does not qualify as an index admission if the **primary reason** for the index admission was related to cancer, rehabilitation, or mental health. These categories are defined in the HWR Cohort Exclusion terminology dataset (terminology dataset #3). In order to use this dataset, first map all **primary** ICD-10-CM codes to the Custom CCS mapping dataset and then lookup whether each inpatient admission had any of these CCS condition categories.

Inpatient admissions that **do not** map to any of these CCS condition categories pass this criteria for inclusion as an index admission.

In [82]:
%%sql
drop table if exists datathon.step_1_2_excl_cr ;
create table datathon.step_1_2_excl_cr as
with hadm_id_excl as (
    select distinct ha.hadm_id
    from hosp.admissions ha 
        inner join datathon.hosp_diagnoses dh on ha.hadm_id = dh.hadm_id and dh.seq_num = 1
        inner join tuva_readm.readmissions__exclusion_ccs_diagnosis_category tr
            on tr.ccs_diagnosis_category = dh.ccs_diagnosis_category
) 
select ds.*
from datathon.step_1_1_incl_cr ds
left join hadm_id_excl on hadm_id_excl.hadm_id = ds.hadm_id
where hadm_id_excl.hadm_id is null

,Success


#### 1.3 Discharged Alive and Not Against Medical Advice
In order to qualify as an index admission the inpatient admission must not have a discharge status of 20 (‘expired’) or 07 (‘left against medical advice’).

In [91]:
%%sql

drop table if exists datathon.step_1_3_disch_loc ;
create table datathon.step_1_3_disch_loc as

with hadm_id_excl as (
select hadm_id 
from hosp.admissions 
where discharge_location in ('20', '07')
)
select ds.*
from datathon.step_1_2_excl_cr ds
left join hadm_id_excl on hadm_id_excl.hadm_id = ds.hadm_id
where hadm_id_excl.hadm_id is null


,Success


#### 1.4 Not a Transfer
Inpatient admissions that result in the patient being transferred to another acute care facility do not count as index admissions. However, the subsequent transfer does count as an index admission if:

The patient is discharged alive and not against medical advice
The patient is discharged to a non-acute care setting (e.g. home, SNF, etc.)
Importantly, the transfer does not necessarily have to meet the other index admission criteria. For example, the transfer does not have to pass the cohort inclusion algorithm as long as the initial admission does.

A transfer is defined as occurring if a patient is discharged from an acute care hospital and admitted to another acute care hospital on the same day or the following day. No other criteria such as discharge status or admit source is required to define a transfer.

In [95]:
%%sql

-- ACUTE HOSPITAL	02, 81, 86 for discharge_location from https://mimic.mit.edu/docs/iv/modules/hosp/admissions/

drop table if exists datathon.step_1_4_transfer ;
create table datathon.step_1_4_transfer as

with hadm_id_excl as (
select hadm_id 
from hosp.admissions 
where discharge_location in ('02', '81', '86')
)
select ds.*
from datathon.step_1_3_disch_loc ds
left join hadm_id_excl on hadm_id_excl.hadm_id = ds.hadm_id
where hadm_id_excl.hadm_id is null





,Success


#### 1.5 Not a Same Day Admission

Inpatient admissions do not qualify as an index admission if the patient has a readmission to the same hospital on the same day as they were discharged for the previous admission for the same condition. In this case the readmission does however qualify as an index admission.


In [112]:
%%sql
-- if the patient has two admission on the same day, it will be two different primary diangosis, 
-- thus just use hadm_id 

drop table if exists datathon.step_1_5_same_day_adm ;
create table datathon.step_1_5_same_day_adm as
with adm_icd as (
select ds.*, dh.icd_code, date_trunc('day', admittime) as admin_day, 
    rank() over(partition by ds.subject_id, admin_day order by admittime) as day_rank
from datathon.step_1_4_transfer ds
inner join datathon.hosp_diagnoses dh on ds.hadm_id = dh.hadm_id and dh.seq_num = 1
),
find_subid_day as (
select subject_id, admin_day
from adm_icd
group by subject_id, admin_day
having count(hadm_id) > 1
),
hadm_id_excl as (
select adm_icd.hadm_id
from find_subid_day join adm_icd on adm_icd.subject_id = find_subid_day.subject_id 
    and adm_icd.admin_day = find_subid_day.admin_day
where day_rank = 1
)
select ds.*, 1 as index_admit_flag
from datathon.step_1_4_transfer ds
left join hadm_id_excl on hadm_id_excl.hadm_id = ds.hadm_id
where hadm_id_excl.hadm_id is null



,Success


#### 1.6 Run-out

In order to qualify as an index admission, the admission must occur at least 30 days before the last date of discharge in the dataset.

For example, suppose your dataset contains inpatient admissions that occurred in calendar year 2018. Without this rule, admissions in this dataset that occurred on the last date in the dataset (i.e. December 31, 2018) could be flagged as index admissions but would never have an associated readmission because the data does not exist. As a result the readmission measure for the month of December 2018 would be artificially low (there would be a typical number of index admissions but fewer than typical readmissions simply because the data does not exist).

**we cannot identify this info in MIMIC-IV, which is a limitation**

### 2.Planned Admission Algorithm

The planned readmission algorithm is used to exclude planned admissions from being flagged as readmissions in the Hospital-wide Measure. Terminology datasets #4-7 are used in this algorithm.

Here are the steps to implement the algorithm:

1. This algorithm requires mapping the **primary** ICD-10-CM code to the custom CCS diagnosis categories and all the ICD-10-PCS codes to the custom CCS procedure categories for each inpatient admission (terminology datasets #8 and #9).

2. For each inpatient admission, check whether the CCS diagnosis **and** procedure categories fall under the always planned list of diagnoses and procedures (terminology datasets #4 and #5). If any match occurs, this is a planned admission.

3. For each inpatient admission, check whether the CCS procedure categories **and** ICD-10-PCS codes fall under the list of potentially planned procedures (terminology dataset #6). If any match occurs, check whether this patient had any **primary** ICD-10-CM code or CCS diagnosis category that is considered always acute (terminology dataset #7). If the answer is no, this is a planned admission.

4. Inpatient admissions that do not qualify as planned admissions based on steps #2 and #3 above are considered unplanned.

In [182]:
%%sql

drop table if exists datathon.step_2_planned_flag ;
create table datathon.step_2_planned_flag as

with always_planned as (
select distinct ds.hadm_id
from datathon.step_1_5_same_day_adm ds 
    inner join datathon.hosp_diagnoses dh on ds.hadm_id = dh.hadm_id and dh.seq_num = 1
    join tuva_readm.readmissions__always_planned_ccs_diagnosis_category tr 
        on dh.ccs_diagnosis_category = tr.ccs_diagnosis_category
union
select distinct ds.hadm_id
from datathon.step_1_5_same_day_adm ds 
    inner join datathon.hosp_procedures dh on ds.hadm_id = dh.hadm_id
    join tuva_readm.readmissions__always_planned_ccs_procedure_category tr 
        on dh.ccs_procedure_category = tr.ccs_procedure_category
),
potential_planned as (
select ds.*
from datathon.step_1_5_same_day_adm ds 
    inner join datathon.hosp_procedures dh on ds.hadm_id = dh.hadm_id
    join tuva_readm.readmissions__potentially_planned_ccs_procedure_category tr 
        on dh.ccs_procedure_category = tr.ccs_procedure_category
union
select ds.*
from datathon.step_1_5_same_day_adm ds 
    inner join datathon.hosp_procedures dh on ds.hadm_id = dh.hadm_id
    join tuva_readm.readmissions__potentially_planned_icd_10_pcs tr 
        on dh.icd_code = tr.icd_10_pcs
),
potential_planned_acute as (
select distinct pp.hadm_id
from potential_planned pp
    inner join datathon.hosp_diagnoses dh on pp.hadm_id = dh.hadm_id and dh.seq_num = 1
    join tuva_readm.readmissions__acute_diagnosis_ccs tr 
        on dh.ccs_diagnosis_category = tr.ccs_diagnosis_category
union
select distinct pp.hadm_id
from  potential_planned pp
    inner join datathon.hosp_diagnoses dh on pp.hadm_id = dh.hadm_id and dh.seq_num = 1
    join tuva_readm.readmissions__acute_diagnosis_icd_10_cm tr 
        on dh.icd_code = tr.icd_10_cm
),
potential_planned_not_acute as (
select distinct pp.hadm_id
from potential_planned pp
left join potential_planned_acute ppa on pp.hadm_id = ppa.hadm_id
where ppa.hadm_id is null    
),
planned as (
select *, 1 as planned
from always_planned
union
select *, 1 as planned
from potential_planned_not_acute
)

select ds.*, coalesce(planned, 0) as planned_flag
from datathon.step_1_5_same_day_adm ds
left join planned on planned.hadm_id = ds.hadm_id

,Success


### 3 Unplanned Readmission Algorithm and Adding social worker flag and demographics, primary icd dx and ccs dx code


An admission that occurs within 30 days of an index admission is considered a readmission if it meets the criteria below:

- Not a Planned Readmission
- Does Not Follow a Planned Readmission
- Not a Multiple Readmission

Planned readmissions are excluded from the readmission measure, as described in the previous section.

In a chain of readmissions, where the planned readmission occurs before an unplanned readmission, the unplanned readmission does not qualify as a readmission.

![](https://thetuvaproject.com/assets/images/unplanned_readmission_1-8c4a21f47f4ce4ed9c59e883ec8ebfb3.png)

In a chain of readmissions, where two or more unplanned readmissions follow an index admission, only the first unplanned readmission qualifies as a readmission. This is because the readmission flag is binary.
![](https://thetuvaproject.com/assets/images/unplanned_readmission_2-74e0f488d788a797209be7f645ba8f88.png)

In [5]:
%%sql
drop table if exists datathon.step_3_readmit ;
create table datathon.step_3_readmit as

with readmit_flag as (
select *, 
    date_trunc('day', admittime) as admit_day,
    date_trunc('day', dischtime ) as disch_day,
    lead(admit_day) over(partition by subject_id order by admittime) - disch_day as days_to_next_adm_from_disch,
    lead(planned_flag) over(partition by subject_id order by admittime) as planned_next_flag,
    case 
        when days_to_next_adm_from_disch <= 30 and planned_next_flag = 0 then 1
        else 0
    end as readmit_flag
from datathon.step_2_planned_flag
),
-- check if make sense

-- select distinct a.*
-- from readmit_flag a join readmit_flag b on a.subject_id = b.subject_id
-- where b.readmit_flag = 1
-- order by a.subject_id, a.admittime
-- limit 20

social_work_flag as (
select distinct hadm_id, 1 as social_work_flag
from hosp.poe
where order_subtype = 'Social Work'
),
-- some do has social work consult more than once

cohort as (
select rf.*, 
    coalesce(social_work_flag, 0) as social_work_flag,
    pa.gender,
    pa.anchor_age + DATE_DIFF('year',rf.admit_day, CAST(concat(CAST(pa.anchor_year AS VARCHAR), '-01-21') as DATE)) AS age, -- noqa: L016
    pa.dod, 
    case 
        when race like '%WHITE%' then 'WHITE'
        when race like '%BLACK%' then 'BLACK'
        when race like '%HISPANIC%' then 'HISPANIC'
        when race like '%ASIAN%' then 'ASIAN'
        else 'OTHER'
    end as race_cat, 
    dh.icd_code as primary_icd_code,
    dh.icd_10_desc, 
    dh.ccs_diagnosis_category as primary_ccs_diagnosis_category,
    dh.ccs_description

from readmit_flag rf
    left join social_work_flag sf on rf.hadm_id = sf.hadm_id
    left join hosp.patients pa on rf.subject_id = pa.subject_id
    left join datathon.hosp_diagnoses dh on rf.hadm_id = dh.hadm_id and dh.seq_num = 1
)
select *
from cohort






,Success


### 4 Charlson comorbidity index

In [6]:
%%sql

drop table if exists datathon.step_4_charlson ;
create table datathon.step_4_charlson as
WITH diag AS (
    SELECT
        hadm_id
        , CASE WHEN icd_version = 9 THEN icd_code ELSE NULL END AS icd9_code
        , CASE WHEN icd_version = 10 THEN icd_code ELSE NULL END AS icd10_code
    FROM hosp.diagnoses_icd
)

, com AS (
    SELECT
        ad.hadm_id

        -- Myocardial infarction
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('410', '412')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I21', 'I22')
            OR
            SUBSTR(icd10_code, 1, 4) = 'I252'
            THEN 1
            ELSE 0 END) AS myocardial_infarct

        -- Congestive heart failure
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '428'
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN ('39891', '40201', '40211', '40291', '40401', '40403'
                , '40411', '40413', '40491', '40493')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4254' AND '4259'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I43', 'I50')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('I099', 'I110', 'I130', 'I132', 'I255', 'I420'
                  , 'I425', 'I426', 'I427', 'I428', 'I429', 'P290'
            )
            THEN 1
            ELSE 0 END) AS congestive_heart_failure

        -- Peripheral vascular disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('440', '441')
            OR
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('0930', '4373', '4471', '5571', '5579', 'V434')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4431' AND '4439'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I70', 'I71')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I731', 'I738', 'I739', 'I771', 'I790'
                                         , 'I792'
                                         , 'K551'
                                         , 'K558'
                                         , 'K559'
                                         , 'Z958'
                                         , 'Z959'
            )
            THEN 1
            ELSE 0 END) AS peripheral_vascular_disease

        -- Cerebrovascular disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '430' AND '438'
            OR
            SUBSTR(icd9_code, 1, 5) = '36234'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('G45', 'G46')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'I60' AND 'I69'
            OR
            SUBSTR(icd10_code, 1, 4) = 'H340'
            THEN 1
            ELSE 0 END) AS cerebrovascular_disease

        -- Dementia
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '290'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('2941', '3312')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('F00', 'F01', 'F02', 'F03', 'G30')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('F051', 'G311')
            THEN 1
            ELSE 0 END) AS dementia

        -- Chronic pulmonary disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '490' AND '505'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('4168', '4169', '5064', '5081', '5088')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J40' AND 'J47'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J60' AND 'J67'
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I278', 'I279', 'J684', 'J701', 'J703')
            THEN 1
            ELSE 0 END) AS chronic_pulmonary_disease

        -- Rheumatic disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '725'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('4465', '7100', '7101', '7102', '7103'
                                        , '7104', '7140', '7141', '7142', '7148'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('M05', 'M06', 'M32', 'M33', 'M34')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('M315', 'M351', 'M353', 'M360')
            THEN 1
            ELSE 0 END) AS rheumatic_disease

        -- Peptic ulcer disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('531', '532', '533', '534')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('K25', 'K26', 'K27', 'K28')
            THEN 1
            ELSE 0 END) AS peptic_ulcer_disease

        -- Mild liver disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('570', '571')
            OR
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('0706', '0709', '5733', '5734', '5738', '5739', 'V427')
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN ('07022', '07023', '07032', '07033', '07044', '07054')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('B18', 'K73', 'K74')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('K700', 'K701', 'K702', 'K703', 'K709', 'K713'
                  , 'K714', 'K715', 'K717', 'K760', 'K762'
                  , 'K763', 'K764', 'K768', 'K769', 'Z944')
            THEN 1
            ELSE 0 END) AS mild_liver_disease

        -- Diabetes without chronic complication
        , MAX(CASE WHEN
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('2500', '2501', '2502', '2503', '2508', '2509')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('E100', 'E101', 'E106', 'E108', 'E109', 'E110', 'E111'
                  , 'E116'
                  , 'E118'
                  , 'E119'
                  , 'E120'
                  , 'E121'
                  , 'E126'
                  , 'E128'
                  , 'E129'
                  , 'E130'
                  , 'E131'
                  , 'E136'
                  , 'E138'
                  , 'E139'
                  , 'E140'
                  , 'E141', 'E146', 'E148', 'E149')
            THEN 1
            ELSE 0 END) AS diabetes_without_cc

        -- Diabetes with chronic complication
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 4) IN ('2504', '2505', '2506', '2507')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('E102', 'E103', 'E104', 'E105', 'E107', 'E112', 'E113'
                  , 'E114'
                  , 'E115'
                  , 'E117'
                  , 'E122'
                  , 'E123'
                  , 'E124'
                  , 'E125'
                  , 'E127'
                  , 'E132'
                  , 'E133'
                  , 'E134'
                  , 'E135'
                  , 'E137'
                  , 'E142'
                  , 'E143', 'E144', 'E145', 'E147')
            THEN 1
            ELSE 0 END) AS diabetes_with_cc

        -- Hemiplegia or paraplegia
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('342', '343')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('3341', '3440', '3441', '3442'
                                        , '3443', '3444', '3445', '3446', '3449'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('G81', 'G82')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('G041', 'G114', 'G801', 'G802', 'G830'
                                         , 'G831'
                                         , 'G832'
                                         , 'G833'
                                         , 'G834'
                                         , 'G839'
            )
            THEN 1
            ELSE 0 END) AS paraplegia

        -- Renal disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('582', '585', '586', 'V56')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('5880', 'V420', 'V451')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '5830' AND '5837'
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN (
                '40301'
                , '40311'
                , '40391'
                , '40402'
                , '40403'
                , '40412'
                , '40413'
                , '40492'
                , '40493'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('N18', 'N19')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I120', 'I131', 'N032', 'N033', 'N034'
                                         , 'N035'
                                         , 'N036'
                                         , 'N037'
                                         , 'N052'
                                         , 'N053'
                                         , 'N054'
                                         , 'N055'
                                         , 'N056'
                                         , 'N057'
                                         , 'N250'
                                         , 'Z490'
                                         , 'Z491'
                                         , 'Z492'
                                         , 'Z940'
                                         , 'Z992'
            )
            THEN 1
            ELSE 0 END) AS renal_disease

        -- Any malignancy, including lymphoma and leukemia,
        -- except malignant neoplasm of skin.
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '140' AND '172'
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '1740' AND '1958'
            OR
            SUBSTR(icd9_code, 1, 3) BETWEEN '200' AND '208'
            OR
            SUBSTR(icd9_code, 1, 4) = '2386'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('C43', 'C88')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C00' AND 'C26'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C30' AND 'C34'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C37' AND 'C41'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C45' AND 'C58'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C60' AND 'C76'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C81' AND 'C85'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C90' AND 'C97'
            THEN 1
            ELSE 0 END) AS malignant_cancer

        -- Moderate or severe liver disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 4) IN ('4560', '4561', '4562')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '5722' AND '5728'
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('I850', 'I859', 'I864', 'I982', 'K704', 'K711'
                  , 'K721', 'K729', 'K765', 'K766', 'K767')
            THEN 1
            ELSE 0 END) AS severe_liver_disease

        -- Metastatic solid tumor
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('196', '197', '198', '199')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('C77', 'C78', 'C79', 'C80')
            THEN 1
            ELSE 0 END) AS metastatic_solid_tumor

        -- AIDS/HIV
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('042', '043', '044')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('B20', 'B21', 'B22', 'B24')
            THEN 1
            ELSE 0 END) AS aids
    FROM datathon.step_3_readmit ad
    LEFT JOIN diag
        ON ad.hadm_id = diag.hadm_id
    GROUP BY ad.hadm_id
)

, ag AS (
    SELECT
        hadm_id
        , age
        , CASE WHEN age <= 50 THEN 0
            WHEN age <= 60 THEN 1
            WHEN age <= 70 THEN 2
            WHEN age <= 80 THEN 3
            ELSE 4 END AS age_score
    FROM datathon.step_3_readmit
),
charlson as( 
SELECT
    ad.subject_id
    , ad.hadm_id
    , ag.age_score
    , myocardial_infarct
    , congestive_heart_failure
    , peripheral_vascular_disease
    , cerebrovascular_disease
    , dementia
    , chronic_pulmonary_disease
    , rheumatic_disease
    , peptic_ulcer_disease
    , mild_liver_disease
    , diabetes_without_cc
    , diabetes_with_cc
    , paraplegia
    , renal_disease
    , malignant_cancer
    , severe_liver_disease
    , metastatic_solid_tumor
    , aids
    -- Calculate the Charlson Comorbidity Score using the original
    -- weights from Charlson, 1987.
    , age_score
    + myocardial_infarct + congestive_heart_failure
    + peripheral_vascular_disease + cerebrovascular_disease
    + dementia + chronic_pulmonary_disease
    + rheumatic_disease + peptic_ulcer_disease
    + GREATEST(mild_liver_disease, 3 * severe_liver_disease)
    + GREATEST(2 * diabetes_with_cc, diabetes_without_cc)
    + GREATEST(2 * malignant_cancer, 6 * metastatic_solid_tumor)
    + 2 * paraplegia + 2 * renal_disease
    + 6 * aids
    AS charlson_comorbidity_index, 
    case 
        when charlson_comorbidity_index > 0 then 1
        else 0
    end as charlson_comorbidity_index_flag
FROM datathon.step_3_readmit ad
LEFT JOIN com
    ON ad.hadm_id = com.hadm_id
LEFT JOIN ag
    ON com.hadm_id = ag.hadm_id
)

-- select the final dataset columns 
select 
    *, coalesce(charlson_comorbidity_index, 0) as charlson_comorbidity_index
from datathon.step_3_readmit ds 
    left join charlson on ds.hadm_id = charlson.hadm_id



,Success


### export csv

In [11]:
### 5 Export to csv for further analysis in R

# init
import duckdb
import pandas as pd
pd.options.display.max_rows = 4000

vars_chosen = '''subject_id,hadm_id,age, gender, race_cat, insurance, 
               primary_icd_code, icd_10_desc, primary_ccs_diagnosis_category,ccs_description,
               charlson_comorbidity_index, charlson_comorbidity_index_flag, social_work_flag, readmit_flag'''

con = duckdb.connect('MIMIC_IV.duckdb')
main_df = con.sql(f"SELECT {vars_chosen} from datathon.step_4_charlson").df()
main_df.to_csv('main_df.csv', index=False)